# Spatial annotation of great apes data 

In [1]:
%matplotlib notebook

import anndata as ad

import napari

import numpy as np
import seaborn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import pathlib


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



from shapely.geometry import MultiLineString, box, Point, MultiPolygon
from shapely.ops import unary_union, polygonize

import shapely




from  geojson.geometry import GeometryCollection
import geojson

import json
from shapely.geometry import Polygon
import pandas as pd

In [2]:

h5adpath = pathlib.Path("/home/imaging_mfish/surveyNAS05/scratch/human/Healthy_MTG_V1/Trygve_MTG_V1/results/Great_apes_72722.h5ad")
datadir = h5adpath.parent

In [3]:
# all sections:
all_sections = ad.read_h5ad(h5adpath)


In [9]:
all_sections

AnnData object with n_obs × n_vars = 153869 × 140
    obs: 'cluster', 'subclass', 'neighborhood', 'class', 'merscope', 'avg.cor', 'genes_detected', 'total_reads', 'prob', 'volume', 'section', 'filename'
    uns: 'gene_panel', 'merscope', 'min_genes', 'min_total_reads', 'min_vol', 'section', 'species', 'upper_bound_area', 'upper_bound_reads', 'upper_genes_read'
    obsm: 'X_umap', 'blanks', 'spatial', 'spatial_cirro'

In [7]:
sections = all_sections.obs["filename"].unique()

In [10]:
subclasses = list(all_sections.obs.subclass.unique() )

layer_groups = {layer_name: [subclass for subclass in subclasses if layer_name in subclass] for layer_name in ["L2/3","L4","L5","L6"]}

In [11]:
random_colors = np.clip(np.random.rand(1000,4)+0.2, a_max=1.0, a_min=0)
random_colors[:,3]=1.0

In [31]:

def add_section_to_napari(viewer, section_anndata,layer_info, colors, spatial_to_use= "spatial", group_column = "subclass", data_name = "data name"):
    yx_data = section_anndata.obsm[spatial_to_use].copy()[:,::-1]
    # don't forget about this:  sign switch instead of axis flip in napari. check for current recommendations
    yx_data[:,0]= -yx_data[:,0]

    viewer.add_points(data = yx_data[:,:], size=20, face_color=[0.4,.4,.4,1],edge_color=[.4,.4,.4,1], name = data_name+" all cells")

    for ii,layer_name in enumerate(layer_info):
        yx_data_layer = section_anndata.obsm[spatial_to_use][section_anndata.obs[group_column].isin(layer_info[layer_name])].copy()[:,::-1]


        # don't forget about this:  sign switch instead of axis flip in napari. check for current recommendations
        yx_data_layer[:,0]= -yx_data_layer[:,0]


        viewer.add_points(data = yx_data_layer[:,:], size=25, face_color=colors[ii,:], edge_color=colors[ii,:], name=layer_name)



In [25]:
#get last layer, it has a single polygon annotation plus a pia-white-matter line.   save it to geojson file
def save_last_polygon_and_line(viewer, savepath, save_screenshot= True):
    
    tgc = GeometryCollection()
    n_polys = len(viewer.layers[-1].data)
    viewer.layers[-1].edge_color = ["black" for b in range(n_polys)]
    viewer.layers[-1].edge_width = [10 for b in range(n_polys)]

    for poly_index in range(n_polys):
        tp = viewer.layers[-1].data[poly_index].copy()
        coordinates = [(tp[r,0],tp[r,1]) for r in range(tp.shape[0])]
        
        if poly_index ==0 :

            tgc.geometries.append(geojson.geometry.Polygon(coordinates = coordinates))
        else:
            tgc.geometries.append(geojson.geometry.LineString(coordinates=coordinates))
        
    with open(savepath, 'w') as w:
        geojson.dump(tgc, w)
    if save_screenshot:
        v.window.screenshot(savepath.parent.joinpath(savepath.stem+"_screenshot.png"))

    return tgc

In [26]:


def cells_in_polygon(cell_locations, polygon,
                     checkBoundingBox = False):
    """
    checks numpy array of point locations and determines if they're inside the polygon 
    currently only checks in 2D
    uses shapely's contains() method

    Args:
        cell_locations is a numpy array size n_cells X 2  
        polygon is a  shapely.geometry.Polygon
        checkBoundingBox if True, use bounding box coordinates to avoid making a containsPoint call.

    Returns:
        1D numpy array of Booleans indicating if the cells are in the polygon
    """
    

    if checkBoundingBox:

        polySpots =[]
        bb_bounds = polygon.bounds
        conditionA = np.logical_or(cell_locations[:,0]< bb_bounds[0] ,
                         cell_locations[:,0]> bb_bounds[2])
        conditionB = np.logical_or(cell_locations[:,1]< bb_bounds[1] ,
                         cell_locations[:,1]> bb_bounds[3]) 

        conditions = np.logical_not( np.logical_or(conditionA, conditionB))
        # this is the subset of spots within the xy bounding box of the polygon:


        okSpots = np.where(conditions)[0]
    else:
        
        okSpots = range(cell_locations.shape[0])

        
    spotList = []

    for spot_row in okSpots:
        spotList.append( Point(cell_locations[spot_row,0:2]))
            
    
    is_in_roi = [polygon.contains(spot) for spot in spotList]
    
                                        

    output = np.zeros(cell_locations.shape[0],dtype=bool)
    output[okSpots] = is_in_roi
                            
                                        
    return output


In [14]:
v = napari.Viewer()

/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to unbind an action which does not exists (napari:also_select_layer_above), this may have no effects. This can happen if your settings are out of date, if you upgraded napari, upgraded or deactivated a plugin, or made a typo in in your custom keybinding.
  action_manager.unbind_shortcut(action)
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to unbind an action which does not exists (napari:also_select_layer_below), this may have no effects. This can happen if your settings are out of date, if you upgraded napari, upgraded or deactivated a plugin, or made a typo in in your custom keybinding.
  action_manager.unbind_shortcut(action)
/home/imaging_mfish/anaconda3/envs/merscope_notebooks/lib/python3.9/site-packages/napari/_qt/qt_viewer.py:344: UserWarning: Attempting to 

# The following cells can be used to cycle through each section, annotate the selected region as a polygon + a line from pia-white matter and save the results

In [35]:
datadir.joinpath("selection_polygon_info").mkdir()
ii =0
all_sections.uns["selection_polygons"] ={}


In [32]:
# clear out anything in your Napari viewer
for layer_number in range(len(v.layers)):
    v.layers.pop()

In [47]:
# plot the data for section ii.
sname = sections[ii]

print(sname)
print(ii)
s_anndata = all_sections[all_sections.obs["filename"]==sname]

add_section_to_napari(v, s_anndata, layer_groups, colors = random_colors)


IndexError: index 6 is out of bounds for axis 0 with size 6

# after running this cell ^^^,  use Napari to add a Shapes layer and add 
- a polygon outlining a uniform piece of cortex 
- a line going from pia to white matter

# Then run the cell below to save the data

In [46]:
gc = save_last_polygon_and_line(v,datadir.joinpath("selection_polygon_info").joinpath( sname+"_cortex_column_polygon.geojson" ))
gc_string = geojson.dumps(gc)
all_sections.uns["selection_polygons"].update( {sname:gc_string})
for layernumber in range(len(v.layers)):
    v.layers.pop()
ii +=1

#  repeat these last 2 cells until all sections have been annotated !

# now saves this data back into the anndata object:
- go through the anndata object, identify the cells within the polygon annotations for each section and add a column in the anndata.obs dataframe


In [49]:
#first, the spatial_cirro coordinates need to be added to the anndata.obs
all_sections.obs["napari_x"] = -all_sections.obsm["spatial"][:,1]
all_sections.obs["napari_y"] = all_sections.obsm["spatial"][:,0]
all_sections.obs["selected_cells"] = False
all_sections.obs["depth_coordinate"] = -1.  # not sure if NaN would be better here... 
all_sections.obs["normalized_depth"] = -0.1  


for ii, sname in enumerate(sections):


    s_anndata = all_sections[all_sections.obs["filename"]==sname]
    if sname not in s_anndata.uns["selection_polygons"].keys():
        # this wasn't annotated with a polygon, meaning there was no selected cells
        print(sname+"    not in keys?")
        continue
    
    gc =  geojson.loads(s_anndata.uns["selection_polygons"][sname])["geometries"]
    for poly in gc:
        if len(poly["coordinates"])<4:
            continue
            
        cell_is_in_selection = cells_in_polygon(s_anndata.obs.loc[:,["napari_x","napari_y" ]].values, shapely.geometry.Polygon(poly["coordinates"]), checkBoundingBox = True)

    
        all_sections.obs.loc[all_sections.obs["filename"]==sname, "selected_cells"] = cell_is_in_selection | all_sections.obs.loc[all_sections.obs["filename"]==sname, "selected_cells"]
        print(sname+": "+str(cell_is_in_selection.sum()))
    # coordinates of selected cells:
    selected_cell_mask = np.logical_and(all_sections.obs["filename"]==sname, all_sections.obs["selected_cells"])
    selected_cell_coordinates = all_sections.obs.loc[selected_cell_mask, ["napari_x","napari_y"]].values
    
    line_array = np.array(gc[1]["coordinates"])
    depth_unit_vector  = (np.diff(line_array, axis = 0)[0,:])
    depth_unit_vector = depth_unit_vector/np.linalg.norm(depth_unit_vector)
    # align the coordinates so the origin is at the beginning of the depth vector:
    sccdiff = np.column_stack([selected_cell_coordinates[:,0]-line_array[0,0], selected_cell_coordinates[:,1]-line_array[0,1]])
    # and dot with the unit vector to get the depth coordinate
    depth = np.dot(sccdiff, depth_unit_vector)
    all_sections.obs.loc[selected_cell_mask,"depth_coordinate"] = depth
    all_sections.obs.loc[selected_cell_mask,"normalized_depth"] = depth/np.max(depth)
    
    
    

202112021529_H1930001Cx46MTG202007203_VMSC01001: 5366
202112021532_H1930001Cx46MTG202007204_vmsc00401: 4209
202112101342_H1930001Cx46MTG0202007205_VMSC01001: 9995
202201271619_H1930002Cx46MTG202007102_VMSC01001: 4852
202201271624_H1930002Cx46MTG202007105_vmsc00401: 5680
202201281113_H1930002Cx46MTG202007104_vmsc00401: 6757


In [51]:
# now write this entire thing back out to a new h5ad file..
all_sections.write_h5ad(datadir.joinpath(h5adpath.stem+"_annotated_with_depth.h5ad"))